In [1]:
import pandas as pd
import random
import math
import hashlib
from collections import Counter

In [2]:
df = pd.read_csv('training_input.csv')
df = df.fillna('NaN')

In [3]:
import math
def is_prime(n):
    if n % 2 == 0 and n > 2: 
        return False
    return all(n % i for i in range(3, int(math.sqrt(n)) + 1, 2))


primes = [i for i in range(2, 50) if is_prime(i)]

In [4]:
def get_str_hash(x, divisor):
    m = hashlib.md5()
    m.update(x.encode("utf-8"))
    h = m.hexdigest()
    number = int(h, 16)
    return number % divisor

In [5]:
for i in primes:
    df['cmd_hash_dv' + str(i)] = df.command_line.apply(lambda x: get_str_hash(x, i))

In [6]:
def entropy(s):
    """Compute the Shannon entropy of string s"""
    counter = Counter(s)
    nchars = float(len(s))
    return -sum(count / nchars * math.log(count / nchars, 2) for count in counter.values())

In [7]:
df = df.drop(columns=['command_line_entropy', 'command_line_len'])

In [8]:
df['command_line_entropy'] = df.command_line.apply(lambda x: entropy(x))
df['command_line_len'] = df.command_line.apply(lambda x: len(x) + random.random() if len(x) < 5 else len(x)) 

In [9]:
features = ['cmd_hash_dv' + str(i) for i in primes]
features.extend(['command_line_entropy', 'command_line_len', 'suspicion_score'])

In [10]:
df[features].to_csv('training.csv', index=False)